In [56]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [57]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy, qiskit
import circuit_cutter
import mlrecon_methods as ml
import json

import numpy as np
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, H_distance_dict, total_counts
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [58]:
with open('LiH_toronto_orign.json') as f:
    unmiti_list = json.load(f)
with open('LiH_toronto_check0.json') as f:
    miti_q0_list = json.load(f)
with open('LiH_toronto_check1.json') as f:
    miti_q1_list = json.load(f)
with open('LiH_toronto_check2.json') as f:
    miti_q2_list = json.load(f)
with open('LiH_toronto_check3.json') as f:
    miti_q3_list = json.load(f)

Untimigated distribution

In [59]:
qubits = 4
# list of all possible measurement outcomes (bitstrings)
all_bits = [ "".join(bits) for bits in itertools.product(["0","1"], repeat = qubits) ]

In [60]:
miti_dist_list = [miti_q0_list[-1], miti_q1_list[-1],miti_q2_list[-1],miti_q3_list[-1]]

In [61]:
def load_probs(miti_dist_list, bit_str):
    output_list = []
    for dist in miti_dist_list:
        try:
            miti_count = dist[bit_str]
        except:
            miti_count = 0
        output_list.append(miti_count)
    return output_list

In [62]:
load_probs(miti_dist_list, '0011')

[0.9551543411276088,
 0.9475296661555079,
 0.9367070552565697,
 0.9034945736620934]

In [63]:
#q0 bitflip
load_probs(miti_dist_list, '0010')

[0, 0.03483509488253257, 0.030995361424800597, 0.03588245514028989]

In [64]:
#q1 bitflip
load_probs(miti_dist_list, '0001')

[0.028103532658738682, 0, 0.027404125427155326, 0.050340334917196444]

In [65]:
#q2 bitflip
load_probs(miti_dist_list, '0111')

[0.008674916118436031, 0.008941176450486596, 0, 0.008684069062346087]

In [66]:
#q3 bitflip
load_probs(miti_dist_list, '1011')

[0.00806721009521661, 0.00869406251147285, 0.004893457891474453, 0]

In [67]:
def bit_weight(dist, index):
    #bitwise distribution
    weight_0 = 0
    weight_1 = 0
    for key in dist.keys():
        if key[len(key) - 1 - index] == '0':
            weight_0 += dist[key]
        elif key[len(key) - 1 - index] == '1':
            weight_1 += dist[key]
        else:
            print("Incorrect key value")
    return weight_0, weight_1

In [68]:
def update_dist(unmiti_dist, miti_dist, index):
    Ppost = {}
    w0, w1 = bit_weight(miti_dist, index)
    u_w0, u_w1 = bit_weight(unmiti_dist, index)
    if w0 == 0:
        w0 = 0.0000000000001
        w1 = 0.9999999999999
    if w1 == 0:
        w1 = 0.0000000000001
        w0 = 0.9999999999999
    if u_w0 == 0:
        u_w0 = 0.0000000000001
        u_w1 = 0.9999999999999
    if u_w1 == 0:
        u_w1 = 0.0000000000001
        u_w0 = 0.9999999999999
        
    for key in unmiti_dist.keys():
        if key[len(key) - 1 - index] == '0':
            Ppost[key] = unmiti_dist[key] / u_w0 * (w0)# / w1)
            #print(w0, w1, w0/w1, Ppost[key])
        elif key[len(key) - 1 - index] == '1':
            Ppost[key] = unmiti_dist[key] / u_w1 * (w1)# / w0)
            #print(w0, w1, w1/w0, Ppost[key])
        else:
            print("Incorrect key value")
    return Ppost

In [69]:
def combine_dist(orign_dist, dist_list):
    output_dist = {}
    for key in orign_dist:
        value = orign_dist[key]
        for dist in dist_list:
            value += dist[key]
        output_dist[key] = value
    return output_dist  

In [70]:
def bayesian_reconstruct(unmiti_dist, miti_dist_list, threshold = 0.0001):
    temp_dist = unmiti_dist.copy()
    h_dist = 1
    while h_dist > threshold:
        temp_dist_start = temp_dist.copy()
        ppost = [0] * len(miti_dist_list)
        for i in range(0, len(miti_dist_list)):
            ppost[i] = update_dist(temp_dist, miti_dist_list[i][0], miti_dist_list[i][1])
        temp_dist = combine_dist(temp_dist, ppost)
        temp_dist = norm_dict(temp_dist)
        h_dist = H_distance_dict(temp_dist, temp_dist_start)
        print("H-dist:", h_dist)
    return temp_dist

In [71]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [83]:
full_dist = []
for idx in range(0, len(miti_q0_list)):
    miti_dist_index = [[miti_q0_list[idx],0], [miti_q1_list[idx],1], [miti_q2_list[idx],2], [miti_q3_list[idx],3]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 125.78399962554899
H-dist: 0.004698280885950247
H-dist: 0.003759227018160538
H-dist: 0.0030086506905016826
H-dist: 0.0024083883022905134
H-dist: 0.0019281530014195918
H-dist: 0.0015438407258954437
H-dist: 0.0012362317967870673
H-dist: 0.0009899817568564942
H-dist: 0.0007928295227796075
H-dist: 0.000634972047029257
H-dist: 0.0005085683376257331
H-dist: 0.00040734493403474524
H-dist: 0.0003262814721180064
H-dist: 0.0002613597479505834
H-dist: 0.00020936327564177653
H-dist: 0.00016771707800989553
H-dist: 0.0001343595778429589
H-dist: 0.00010764012526004517
H-dist: 8.623701190661104e-05
H-dist: 125.78399962555167
H-dist: 0.013855499685155438
H-dist: 0.012083422726564316
H-dist: 0.010589136148472384
H-dist: 0.009316491994886102
H-dist: 0.008223329987499537
H-dist: 0.007277499162633615
H-dist: 0.006454123781524456
H-dist: 0.005733683500930447
H-dist: 0.005100645306328875
H-dist: 0.004542478399058888
H-dist: 0.004048938994563288
H-dist: 0.0036115471963937577
H-dist: 0.0032232014164603

In [85]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [86]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [87]:
final_expect_val

(-1.0704399173342194+0j)

In [103]:
#no bayesian recombine:

In [102]:
miti_dist_list = [miti_q0_list, miti_q1_list,miti_q2_list,miti_q3_list]
for idx in range(0, 4):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(miti_dist_list[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-1.0345563346769033+0j)
expectation value for mitigating qubit1:(-1.0578289603343904+0j)
expectation value for mitigating qubit2:(-1.0436437636872364+0j)
expectation value for mitigating qubit3:(-1.023004976849608+0j)


In [109]:
#untimigated result:
unmiti_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        unmiti_expect_val += coeff * evaluation(unmiti_list[i], shots = total_counts(unmiti_list[i]), Pauli = Pauli_tuple[0])

In [110]:
unmiti_expect_val

(-1.0194585815515294+0j)